This file is designed to work with Google Colab; for the data table, download the Excel file(df-ga-channel-group-report-monthly), update it as needed, and then upload and convert it to a Google Spreadsheet.








In [21]:
!pip install japanize-matplotlib
!pip install reportlab
!pip install --upgrade reportlab

In [22]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import japanize_matplotlib
from datetime import datetime, timedelta

# ReportLab で PDF を生成するために使うクラスや関数
from reportlab.lib.pagesizes import A4
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image, PageBreak
)
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from reportlab.lib.units import inch

# Google Colab 用の認証関連
from google.colab import auth, drive
import gspread
from google.auth import default

In [23]:
#===============================================================================
# (1) 基本設定
#===============================================================================
# ここでは、どのスプレッドシートからデータを読み込むかや、
# レポートで扱うチャネル、メトリクスのリストを定義しています。
FILENAME = "df-ga-channel-group-report-monthly"  # Google Sheetsファイル名
SHEET_NAME = "df_monthly_channel"               # シート名

DESIRED_CHANNELS = [
    'Organic Search',
    'Direct',
    'Organic Social',
    'Referral',
    'Unassigned'
]

# 「Sessions」「engagedSessions」の2種類を今回のレポート対象メトリクスとする
METRICS = ['sessions', 'engagedSessions']

In [24]:
#===============================================================================
# (2) Google Sheetsからデータを読み込む準備
#===============================================================================
# Google Drive をマウントして認証
drive.mount('/content/drive', force_remount=True)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

def load_sheet_data(spreadsheet_name, sheet_name):
    """
    指定のスプレッドシートとシート名を受け取り、
    そこから全データをDataFrame化して返す関数
    """
    ss = gc.open(spreadsheet_name)            # スプレッドシートを開く
    worksheet = ss.worksheet(sheet_name)      # 特定のシートを取得
    data = worksheet.get_all_values()         # すべてのセルを文字列として取得
    df = pd.DataFrame(data[1:], columns=data[0])  # 先頭行をカラム名にしてDataFrame生成
    return df

# 実際に呼び出してデータ読み込み
df = load_sheet_data(FILENAME, SHEET_NAME)


Mounted at /content/drive


In [25]:

#===============================================================================
# (3) 前処理用の関数
#===============================================================================
def preprocess_df(df):
    """
    【入力】
      df: シートから読み込んだDataFrame
    【処理内容】
      - 数値データに変換
      - 日付を datetime 型に変換
      - 月・年を抽出
      - さらに会計年度(fiscal_year)を付与
    【出力】
      前処理後の df を返す
    """
    # 変換を除外するカラム一覧（文字列のまま扱いたい、あるいは後で使うから）
    exclude_columns = ['yearMonth', 'sessionDefaultChannelGroup']

    # 上記以外の列はすべて数値に変換する
    for column in df.columns:
        if column not in exclude_columns:
            # 文字列中のカンマを除去して→数値に変換
            df[column] = pd.to_numeric(df[column].astype(str).str.replace(',', ''), errors='coerce')

    # yearMonthを「YYYYMM」形式の文字列とみなし、datetime型に変換
    df['yearMonth'] = pd.to_datetime(df['yearMonth'], format='%Y%m')

    # 年・月をそれぞれ数値化
    df['year'] = df['yearMonth'].dt.year
    df['month'] = df['yearMonth'].dt.month

    # 会計年度を判定して記録する
    # ここでは 6月開始～翌年5月終了を1期とする例
    df['fiscal_year'] = df['yearMonth'].apply(
        lambda x: 'FY23' if x < pd.Timestamp('2023-06-01')
        else 'FY24' if pd.Timestamp('2023-06-01') <= x <= pd.Timestamp('2024-05-31')
        else 'FY25' if pd.Timestamp('2024-06-01') <= x <= pd.Timestamp('2025-05-31')
        else None
    )

    return df

def custom_month_sort(df):
    """
    6月始まりの「会計年度順」でソートを行うための関数。
    たとえばFY24が6月で始まるなら、6→7→8…→5の順で並べたい。
    """
    # 6月を1番目、7月を2番目…5月を12番目と定義
    month_order = {
        6: 1, 7: 2, 8: 3, 9: 4, 10: 5, 11: 6,
        12: 7, 1: 8, 2: 9, 3: 10, 4: 11, 5: 12
    }
    # mapで「並び順の番号」を新列に付与し、それを使ってsortする
    df['sort_key'] = df['month'].map(month_order)
    df = df.sort_values(by='sort_key').drop('sort_key', axis=1)
    return df

# 前処理関数を実行
df = preprocess_df(df)

In [26]:
#===============================================================================
# (4) 各チャネルごとのデータを抜き出す
#===============================================================================
df_hub = {}
for ch in DESIRED_CHANNELS:
    # sessionDefaultChannelGroup列が ch と一致する行だけを取り出す
    df_channel = df[df['sessionDefaultChannelGroup'] == ch].copy()
    df_hub[ch] = df_channel

In [27]:
#===============================================================================
# (5) テーブル/グラフ生成用の関数
#===============================================================================
def generate_monthly_table(metric, df):
    """
    【入力】
      metric: 'Sessions'など計算対象のメトリクス名 (文字列)
      df    : 特定チャネルだけを抜き出したDataFrame
    【処理内容】
      fiscal_year と month単位で合計し、FY24/FY25だけ取り出してピボット
      → YoY, MoM の計算列を追加
      → 最下行に合計(Total)を付与
    【出力】
      ピボット形のDataFrame
    """
    # fiscal_year, month 単位で合計
    df_agg = df.groupby(['fiscal_year', 'month'])[metric].sum().reset_index()

    # ピボットテーブルで月をインデックス、fiscal_yearを列に割り当て
    df_pivot = df_agg.pivot_table(
        index='month',
        columns='fiscal_year',
        values=metric,
        fill_value=0
    ).reset_index()

    # 月を「会計年度の順番」でソート
    df_pivot = custom_month_sort(df_pivot)
    df_pivot.set_index('month', inplace=True)

    # FY23は表示しない例
    if 'FY23' in df_pivot.columns:
        df_pivot.drop(columns='FY23', inplace=True)

    # FY24, FY25が無い場合は列を作って0にしておく
    for fy in ['FY24', 'FY25']:
        if fy not in df_pivot.columns:
            df_pivot[fy] = 0
        # 型はintに揃える
        df_pivot[fy] = df_pivot[fy].astype(int)

    # YoY計算 ( (FY25 / FY24) - 1 ) × 100%
    def yoy_func(row):
        if row['FY24'] == 0 or row['FY25'] == 0:
            return '-'
        return f"{int(round((row['FY25'] / row['FY24'] - 1) * 100))}%"
    df_pivot['YoY'] = df_pivot.apply(yoy_func, axis=1)

    # MoM計算 (FY25の前月比のみを例示)
    df_pivot['MoM'] = df_pivot['FY25'].pct_change() * 100
    df_pivot['MoM'] = df_pivot['MoM'].apply(
        lambda x: '-' if not np.isfinite(x) else f"{int(round(x))}%"
    )

    # 合計行(Total)を作成
    total_fy24 = df_pivot['FY24'].sum()
    total_fy25 = df_pivot['FY25'].sum()
    total_row = pd.DataFrame({
        'FY24': [total_fy24],
        'FY25': [total_fy25],
        'YoY': ['-'],
        'MoM': ['-']
    }, index=['Total'])

    # concatでテーブルに合計行を追加
    df_pivot = pd.concat([df_pivot, total_row])
    return df_pivot

def generate_monthly_graph(metric, df, channel_name, title):
    """
    【入力】
      metric      : 'Sessions'など
      df          : 特定チャネルのDataFrame
      channel_name: チャネル名(グラフ保存ファイル名の一部に使用)
      title       : グラフ上に表示するタイトル
    【処理内容】
      fiscal_year, month 単位で合計 → ピボット化 → FY24, FY25 の棒グラフ
      → 保存してファイルパスを返却
    【出力】
      生成したグラフの画像ファイルパス（文字列）
    """
    # 月ごと＆fiscal_yearごとに合計
    df_agg = df.groupby(['fiscal_year', 'month'])[metric].sum().reset_index()

    # ピボットテーブル化して、FY24/FY25列を作成
    df_pivot = df_agg.pivot_table(
        index='month',
        columns='fiscal_year',
        values=metric,
        fill_value=0
    ).reset_index()

    # 月を会計年度順で並べる
    df_pivot = custom_month_sort(df_pivot)
    df_pivot.set_index('month', inplace=True)

    # FY24, FY25が無い場合は0列を作る
    for fy in ['FY24', 'FY25']:
        if fy not in df_pivot.columns:
            df_pivot[fy] = 0
        df_pivot[fy] = df_pivot[fy].astype(int)

    # グラフ描画
    plt.figure(figsize=(6, 6))
    bar_width = 0.35
    index = np.arange(len(df_pivot.index))

    # FY24の棒グラフを描画。colorを '#6AC1B7' に指定
    plt.bar(index, df_pivot['FY24'], bar_width, color='#6AC1B7', label='FY24')
    # FY25の棒グラフを描画。colorを '#517D99' に指定。棒の位置は少し右にずらす。
    plt.bar(index + bar_width, df_pivot['FY25'], bar_width, color='#264E86', label='FY25')

    # Y軸の数値が指数表記されないように設定
    y_formatter = ScalarFormatter(useOffset=False)
    y_formatter.set_scientific(False)
    plt.gca().yaxis.set_major_formatter(y_formatter)

    # 軸ラベルやタイトル設定
    plt.xlabel('Month')
    plt.ylabel(metric)
    plt.title(title)
    plt.xticks(index + bar_width/2, df_pivot.index)
    plt.legend()
    plt.grid(True, linewidth=0.2, linestyle='--', axis='y')
    plt.tight_layout()

    # 画像ファイルを保存
    image_dir = 'images'
    os.makedirs(image_dir, exist_ok=True)
    channel_sanitized = channel_name.replace(" ", "_")
    image_path = os.path.join(image_dir, f'{channel_sanitized}_{metric}.png')
    plt.savefig(image_path)
    plt.close()

    return image_path

def create_table(data, image_path, styles):
    """
    【入力】
      data       : テーブルに表示したい2次元配列データ
      image_path : 生成したグラフの画像ファイルパス
      styles     : ReportLabのスタイル
    【処理内容】
      - dataをReportLabのTable形式に変換
      - 見た目のスタイル設定を追加
      - 画像オブジェクトを用意（なければ代わりのテキスト）
    【出力】
      (tableオブジェクト, imageオブジェクト) のタプル
    """
    # 数値をカンマ区切りの文字列に変換
    formatted_data = []
    for row in data:
        formatted_row = []
        for val in row:
            if isinstance(val, (int, float)) and not isinstance(val, bool):
                formatted_row.append(f"{int(val):,}")  # 3桁区切り
            else:
                formatted_row.append(val)
        formatted_data.append(formatted_row)

    # Tableオブジェクト生成
    table = Table(formatted_data, repeatRows=1)

    # テーブル全体のスタイル
    table_style = TableStyle([
        # ヘッダー行(最初の行)の背景色・文字色
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#517D99')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),

        # 全セルを中央揃え
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),

        # フォント指定
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 7),

        # セル内部の余白
        ('LEFTPADDING', (0, 0), (-1, -1), 8),
        ('RIGHTPADDING', (0, 0), (-1, -1), 8),
        ('TOPPADDING', (0, 0), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 2),

        # 表の枠線
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black)
    ])
    table.setStyle(table_style)

    # 行や列単位で追加の装飾
    # ここでは「Month」列の背景を少し変える例
    month_col_index = 0  # 左端がMonth列
    table.setStyle(TableStyle([
        ('BACKGROUND', (month_col_index, 1), (month_col_index, -1), colors.whitesmoke)
    ]))

    # 画像（グラフ）オブジェクト
    try:
        img = Image(image_path)
        # 画像サイズを縮小
        img.drawHeight = 260
        img.drawWidth = 260
    except Exception as e:
        # 画像が存在しない場合、代替テキストを表示
        img = Paragraph(f"Image not found: {e}", styles['BodyText'])

    return table, img


In [28]:
#===============================================================================
# (6) PDF 生成関数
#===============================================================================
def generate_pdf(datasets):
    """
    【入力】
      datasets: [(df_table, image_path, title), ...] のリスト
                df_table : テーブルにしたいデータ
                image_path: グラフ画像へのファイルパス
                title    : 見出しに使うタイトル文字列
    【処理内容】
      - カバーページを追加
      - 目次（Table of Contents）を追加
      - 各データセットごとにテーブルと画像を並べてPDFに配置
      - 最後に注意書きや、メトリクスの説明を加えて終了
    """
    current_date = datetime.now().strftime('%Y-%m-%d')

    # SimpleDocTemplate: PDFの「1つのドキュメント」を生成する器
    pdf = SimpleDocTemplate(
        f"ga-channel-group-report-monthly-simple-{current_date}.pdf",
        pagesize=A4,
        leftMargin=0.8*inch,
        rightMargin=0.8*inch,
        topMargin=80,
        bottomMargin=80
    )
    styles = getSampleStyleSheet()
    elements = []

    #----------------------------------------
    # (A) カバーページ
    #----------------------------------------
    first_day_this_month = datetime(datetime.now().year, datetime.now().month, 1)
    last_day_prev_month = first_day_this_month - timedelta(days=1)
    last_day_prev_month_str = last_day_prev_month.strftime('%Y-%m-%d')

    cover_title = Paragraph('GA4 Channel Report - Monthly', styles['Title'])
    cover_title_2 = Paragraph('Default Channel Group', styles['Title'])
    previous_month = datetime.now() - timedelta(days=30)
    cover_date = Paragraph(f'{previous_month.strftime("%B %Y")}', styles['Title'])

    # 位置調整のためSpacerを使用しながら要素を追加
    elements.append(Spacer(1, A4[1]/2 - 180))
    elements.append(cover_title)
    elements.append(cover_title_2)
    elements.append(Spacer(1, 12))
    elements.append(cover_date)
    elements.append(Spacer(1, 200))
    elements.append(Paragraph(
        f'Created by: Shohei on {current_date}',
        styles['BodyText']
    ))
    elements.append(Paragraph(
        'Website: heysho.com',
        styles['BodyText']
    ))
    elements.append(Paragraph(
        'Data Source: GA4 - Default Channel Group',
        styles['BodyText']
    ))
    elements.append(Paragraph(
        f'Data Range: 2023-06-01 - {last_day_prev_month_str}',
        styles['BodyText']
    ))
    elements.append(PageBreak())

    #----------------------------------------
    # (B) 目次(Table of Contents)
    #----------------------------------------
    elements.append(Paragraph('Table of Contents', styles['Heading1']))
    elements.append(Spacer(1, 12))

    for idx, (_, _, title) in enumerate(datasets):
        anchor = f'section_{idx}'
        # クリックすると該当ページにジャンプするリンクを埋め込む
        toc_entry = Paragraph(f'<link href="#{anchor}">{title}</link>', styles['BodyText'])
        elements.append(toc_entry)

    elements.append(Spacer(1, 20))
    elements.append(Paragraph("**Click to jump to the page", styles['Normal']))
    elements.append(PageBreak())

    #----------------------------------------
    # (C) データページ（チャネル別テーブル&グラフ）
    #----------------------------------------
    for idx, (df_table, image_path, title) in enumerate(datasets):
        # index名に"Month"というラベルをつけ、データとして扱う
        df_table = df_table.reset_index()
        df_table.rename(columns={'index': 'Month'}, inplace=True)
        # ReportLabのTableに渡せるように、2次元のリスト化
        data = [df_table.columns.to_list()] + df_table.values.tolist()

        # テーブル・画像のオブジェクトを作成
        table_obj, image_obj = create_table(data, image_path, styles)

        # セクションアンカーを設置
        anchor = f'section_{idx}'
        elements.append(Paragraph(f'<a name="{anchor}"/>{title}', styles['Heading2']))
        elements.append(Spacer(1, 6))

        # テーブルと画像を横並びに配置するため、Tableレイアウトを使う
        col_layout = Table(
            [[table_obj, image_obj]],
            colWidths=[220, 280],
            style=[
                ('ALIGN', (0, 0), (0, 0), 'RIGHT'),
                ('ALIGN', (1, 0), (1, 0), 'LEFT')
            ]
        )
        elements.append(col_layout)
        elements.append(Spacer(1, 20))

        # 2つのチャートごとに改ページする例
        if (idx + 1) % 2 == 0:
            elements.append(PageBreak())

    #----------------------------------------
    # (D) 最終ページ
    #----------------------------------------
    elements.append(Paragraph('Usage Rights and License', styles['Heading3']))
    elements.append(Spacer(1, 6))
    elements.append(Paragraph(
        'The use of this template is restricted to personal purposes only. Any commercial use or provision to third parties is strictly prohibited.Redistribution of the template, as well as the redistribution of any modified version or derivative works that incorporate modifications, is prohibited in all forms.The sale, transfer, or public use (including online sharing) of any part or the entirety of the template is also prohibited.',
        styles['BodyText']
    ))

    elements.append(Spacer(1, 14))

    # メトリクスの説明を追加
    elements.append(Paragraph('Explanation of Metrics', styles['Heading3']))
    elements.append(Spacer(1, 6))

    metrics_explanations = [
        {
            "title": "sessions",
            "description": "Number of user visits to the website during a specified period."
        },
        {
            "title": "engagedSessions",
            "description": "Number of engaged sessions that happen on the website."
        },
        {
            "title": "MoM (Month-over-Month)",
            "description": "Percentage change from one month to the previous month."
        },
        {
            "title": "YoY (Year-over-Year)",
            "description": "Percentage change compared to the same month in the previous fiscal year."
        }
    ]

    for metric in metrics_explanations:
        text = f"<bullet>&bull;</bullet> <b>{metric['title']}</b>: {metric['description']}"
        elements.append(Paragraph(text, styles['BodyText']))
        elements.append(Spacer(1, 6))


    elements.append(Spacer(1, 12))

    # チャネルの説明
    elements.append(Paragraph('Explanation of Channels', styles['Heading3']))
    elements.append(Spacer(1, 6))

    channels_explanations = [
        {
            "title": "All Channel",
            "description": "Represents the total aggregate of all traffic sources combined, providing a holistic view of website performance."
        },
        {
            "title": "Organic Search",
            "description": "Traffic generated from unpaid search engine results, such as Google, Yahoo, Bing, based on user queries."
        },
        {
            "title": "Paid Search",
            "description": "Traffic generated from paid advertisements on search engines, often through platforms like Google Ads."
        },
        {
            "title": "Paid Shopping",
            "description": "Traffic driven by paid product listings on platforms like Google Shopping, P-Max, highlighting specific products."
        },
        {
            "title": "Paid Other",
            "description": "Traffic from other paid advertising campaigns that don't fall into specific categories, such as Line Ads."
        },
        {
            "title": "Display",
            "description": "Traffic from banner advertisements displayed on third-party websites or apps."
        },
        {
            "title": "Paid Social",
            "description": "Traffic generated through paid advertisements on social media platforms, such as Facebook, Instagram, or LinkedIn."
        },
        {
            "title": "Organic Social",
            "description": "Traffic originating from unpaid posts or shares on social media platforms."
        },
        {
            "title": "Email",
            "description": "Traffic driven by email marketing campaigns, such as newsletters or promotional emails."
        },
        {
            "title": "Direct",
            "description": "Traffic from users directly entering the website URL into their browser or using bookmarks."
        },
        {
            "title": "Referral",
            "description": "Traffic referred from other websites through links to the site."
        },
        {
            "title": "Unassigned",
            "description": "Traffic that cannot be attributed to any specific channel due to tracking limitations or missing data."
        }
    ]

    for channel in channels_explanations:
        text = f"<bullet>&bull;</bullet> <b>{channel['title']}</b>: {channel['description']}"
        elements.append(Paragraph(text, styles['BodyText']))
        elements.append(Spacer(1, 6))

    elements.append(PageBreak())

    # PDFを書き出し
    pdf.build(elements)
    print(f"PDF 'ga-channel-monthly-report-{current_date}.pdf' generated successfully.")


In [29]:
#===============================================================================
# (7) メイン処理：チャネル・メトリクスごとにテーブル＆グラフを作ってPDFへ
#===============================================================================
datasets = []
# DESIRED_CHANNELS でループし、各チャネルの df を取り出しながら処理
for idx, ch in enumerate(DESIRED_CHANNELS):
    df_data = df_hub.get(ch, pd.DataFrame())
    if df_data.empty:
        # 該当チャネルのデータがなかったらスキップ
        continue

    # チャネルごとに、SessionsとengagedSessionsの2種を出力
    for metric in METRICS:
        # PDFの目次などに表示するタイトル
        title_suffix = 'a' if metric == 'sessions' else 'b'
        title_number = idx + 1
        report_title = f"{title_number}-{title_suffix}. {ch} - {metric}"

        # 表の作成
        monthly_table = generate_monthly_table(metric, df_data)

        # グラフ画像の作成
        image_path = generate_monthly_graph(metric, df_data, ch, report_title)

        # 最後にまとめて PDF に入れるため、datasetsリストに追加
        datasets.append((monthly_table, image_path, report_title))

# PDFを生成
generate_pdf(datasets)

PDF 'ga-channel-monthly-report-2025-04-13.pdf' generated successfully.
